# Gridded Datasets

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.notebook_extension('bokeh', 'matplotlib')

In the [previous guide](3-Tabular_Datasets.ipynb) we discovered how to work with tabular datasets. Although this is a near ubiquitous type of data many datasets are best represented by n-dimensional arrays whether that is simple images, volumetric data or higher dimensional parameter spaces. On a 2D screen and using traditional plotting libraries it is often difficult to visualize such parameter spaces quickly and succinctly, using HoloViews we can quickly slice and dice such a dataset to quickly explore the data and answer questions about it.

## Gridded

Gridded datasets usually represent observations of some continuous variable across multiple dimensions. This could me everything from a simple image representing luminance values across a 2D surface, volumetric 3D data, an image sequence over time or any other multi-dimensional parameter space. This type of data is particularly common in research areas that make use of spatial imaging or modeling such as climatology, biology and astronomy but can also be used to represent any arbitrary data that varies over multiple dimensions.

In HoloViews terminology the dimensions the data varies over are the so called key dimensions (**kdims**), which define the coordinates of the underlying array. The actual value arrays are described by the value dimensions (**vdims**). Libraries like ``xarray`` or ``iris`` allow you to store the coordinates with the array, but here we will declare the coordinate arrays ourselves so we can get a better understanding of how the gridded data interfaces work. We will therefore start by loading a very simple 3D array:

In [ ]:
data = np.load('../datasets/twophoton.npz')
calcium_array = data['Calcium']
calcium_array.shape

This particular dataset represents imaging data obtained with 2-photon calcium imaging, which provides an indirect measure of neural activity encoded via changes in fluorescent light intensity. The 3D array represents the activity of a 2D imaging plane over time, i.e. it is in effect a sequence of images with a shape of (62, 111) over 50 time steps. Just as we did in the [Tabular Dataset] guide we start by wrapping our data in a HoloViews ``Dataset``. However for HoloViews to understand the raw NumPy array we need to pass coordinates for each of the dimensions (or axes) of the data, here we will simply use integer coordinates for the ``'Time'``, ``'x'`` and ``'y'`` dimensions:

In [ ]:
ds = hv.Dataset((np.arange(50), np.arange(111), np.arange(62), calcium_array),
                kdims=['Time', 'x', 'y'], vdims=['Fluorescence'])
ds

As we should be used to by now the ``Dataset`` repr shows us the dimensions of the data. If we inspect the ``.data`` attribute we can see that by default HoloViews will store this data as a simple dictionary of our key dimension coordinates and value dimension arrays:

In [ ]:
type(ds.data), list(ds.data.keys())

#### Other datatypes

Instead of defining the coordinates manually we recommend using [xarray](http://xarray.pydata.org/en/stable/), which will flexibly work with labeled n-dimensional arrays. We can even make a clone of our dataset set the datatype to xarray to convert to an ``xarray.Dataset``, which is the recommended format:

In [ ]:
ds.clone(datatype=['xarray']).data

To see more details on working with different datatypes have a look at the [Gridded Data] user guide.

#### Viewing the data

Perhaps the most natural representation of this dataset is as an Image displaying the fluorescence at each point in time. So let's just display it that way, using the ``.to`` interface we can map the dimensions of our ``Dataset`` onto the dimensions of an Element. To display an image we will pick the ``Image`` element and specify the ``'x'`` and ``'y'`` as the key dimension, since we only have one value dimension we won't have to declare it explicitly.

In [ ]:
%opts Image (cmap='viridis')
ds.to(hv.Image, ['x', 'y']).hist()

In addition to the slider widget you can select the ``Box select`` tool in the plot toolbar and select a Fluorescence range on the Histogram, which will control the color mapping range. Also try pressing the ``P`` and ``R`` buttons after clicking on the slider to play the animation forward and in reverse respectively. When using ``.to`` or ``.groupby`` on larger datasets you can use the ``dynamic=True`` flag, letting you explore the parameter space dynamically (for more detail have a look at the [Live Data] and [Pipeline] sections).

#### Selecting

Often when working with multi-dimensional datasets we are only interested in small regions of the parameter space. When working with neural imaging data like this it is very common to focus on regions of interest (ROIs) within the larger image. Here we will fetch some bounding boxes the data we loaded earlier. ROIs are often more complex polygons but for simplicity's sake we will use simple rectangular ROIs specified as the left, bottom, right and top coordinate of a bounding box.

In [ ]:
ROIs = data['ROIs']
roi_bounds = hv.NdOverlay({i: hv.Bounds(tuple(roi)) for i, roi in enumerate(ROIs)})
print(ROIs.shape)

Here we have 147 ROIs representing bounding boxes around 147 identified neurons in our data. To display them we have wrapped the data in ``Bounds`` elements, which we can overlay on top of our animation. Additionally we will create some ``Text`` elements to label each ROI. Finally we will use the regular Python indexing semantics to select along the Time dimension, which is the first key dimension and can therefore simply with ``ds[21]``. Just like the ``select`` method this indexes and slices by value not the index (which are one and the same here):

In [ ]:
%%opts Image [width=400 height=400 xaxis=None yaxis=None] 
%%opts Bounds (color='white') Text (text_color='white' text_font_size='8pt')

opts = dict(halign='left', valign='bottom')
roi_text = hv.NdOverlay({i: hv.Text(roi[0], roi[1], str(i), **opts) for i, roi in enumerate(ROIs)})
(ds[21].to(hv.Image, ['x', 'y']) * roi_bounds * roi_text).relabel('Time: 21')

Now we can use these bounding boxes to select some data since they simply represent coordinates. Have a look at the ROI #60 for example, we can see the neuron activate quite strongly in the middle of our animation. Using the ``select`` method we can select the x and y-coordinates of our ROI and the rough time period when we saw the neuron respond.

In [ ]:
x0, y0, x1, y1 = ROIs[60]
roi = ds.select(x=(x0, x1), y=(y0, y1), time=(250, 280)).relabel('ROI #60')
roi.to(hv.Image, ['x', 'y'])

#### Faceting

Even though we have selected a very small region of the data, there is still quite a lot of data there. We can use the ``faceting`` methods to display the data in different ways. Since we have only a few pixels in our dataset now, we can for example plot how the fluorescence changes at each pixel in our ROI over time. We simply use the ``.to`` interface to display the data as ``Curve`` types with time as the key dimension. If you recall from the [Tabular Data](1-Tabular_Data.ipynb), the two method will group by any remaining key dimensions (in this case ``'x'`` and ``'y'``) to display sliders. Here we will instead facet the ``Curve`` elements using the ``.grid``allowing us to see the evolution of the fluorescence signal over time and space:

In [ ]:
%%opts GridSpace [shared_xaxis=True shared_yaxis=True]
roi.to(hv.Curve, 'Time').grid()

#### Aggregating

Instead of generating a Curve for each pixel individually we may instead want to average the data across x- and y- to get a more robust signal. For that purpose we can use the aggregate method to get the average signal within the ROI window, using the ``spreadfn`` we can also compute the standard deviation between pixels. We will display the mean and standard deviation data as a overlay of a ``Spread`` and ``Curve`` Element:

In [ ]:
%%opts Overlay [show_legend=False width=600]
agg = roi.aggregate('Time', np.mean, spreadfn=np.std)
hv.Spread(agg) * hv.Curve(agg)